In [1]:
import scipy
import scipy.io
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat
import os
from feature_extraction import features_estimation
import pandas as pd

In [2]:
# Define the function to plot a graph
def plot_signal_modification(time, original_signal, time1, new_signal):
    """
    Function to plot a simple graph.

    Parameters:
    - x: List or array of x-values
    - y: List or array of y-values
    - title: Title of the graph (default: "Graph")
    - xlabel: Label for the x-axis (default: "X-axis")
    - ylabel: Label for the y-axis (default: "Y-axis")
    """

    # Plotting the original EMG signal
    plt.figure(figsize=(10, 5))
    plt.plot(time, original_signal, color='b', linewidth=2)
    plt.plot(time1, new_signal, color='r', linewidth=1)

    # Adding labels and title
    plt.xlabel("Time (s)")
    plt.ylabel("EMG Signal (mV)")
    plt.title("EMG Signal Over Time")
    plt.grid(True)


    # time_resampled = np.linspace(0, time[-1], target_length)
    
    plt.show()


This function was used to uniformized subject gesture

In [3]:
def uniformize_data_subject(file_path, sub_num):

    num_sensors = 24
    target_length = 15376

    # Load the MATLAB file
    mat_data = scipy.io.loadmat(file_path)
    data = mat_data['subjectsSpecificSetGestures']
    set_of_all_gesture_by_subject = np.array(data)

    gestures = set_of_all_gesture_by_subject.shape[1]
    repetitions = set_of_all_gesture_by_subject.shape[0]


    # we access all repetitions of the 7 types of hand gestures
    for i in range(repetitions): # shape (28,)

        # we access a specific gesture type
        for j in range(gestures):

            # hand_gesture_type contains the electrovolts reading of the 24 sensors placed on the subjects arms at the ith repetition of the jth hand gesture type
            hand_gesture_type = set_of_all_gesture_by_subject[i,j]    # shape(....,24)
            
            # we check if the sampling length of the signals of this gesture matches the desired length
            if hand_gesture_type.shape[0] != target_length:

                # if it isn't we create a new array of the appropriate size
                new_gesture = np.zeros((target_length, num_sensors))

                # for all sensors we modify the expand the length of the gesture recorded
                for k in range(num_sensors):
                    sample_to_alter = hand_gesture_type[:,k]  # shape (..., 1)
                    resampled_signal = signal.resample(sample_to_alter , target_length)
                    new_gesture[:,k] = resampled_signal

                    # time = np.arange(len(sample_to_alter)) / sampling_rate
                    # time2 = np.arange(len(resampled_signal)) / sampling_rate
                    # plot_signal_modification(time, sample_to_alter, time2, resampled_signal)
                
                set_of_all_gesture_by_subject[i,j] = new_gesture

   
    # output_data = {'processed_data': set_of_all_gesture_by_subject}

    # output_dir = "E:\\Chris\\putEMG Project\\processed_data"
    # os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    # output_path = os.path.join(output_dir, 'processed_gestures_subject'+ str(sub_num)+ '.mat')

    # # Save the file
    # savemat(output_path, output_data)
    


In [ ]:
def extract_features(file_path, sub_num, sampling_rate, window_size, step_size):

    # Load the MATLAB file
    mat_data = scipy.io.loadmat(file_path)
    data = mat_data["processed_data"]
    set_of_all_gesture_by_subject = np.array(data)

    print("in the function")

    gestures = set_of_all_gesture_by_subject.shape[1]
    repetitions = set_of_all_gesture_by_subject.shape[0]
    sensors = (set_of_all_gesture_by_subject[0][0].shape)[1]

    print("number of gestures detected is: ", gestures)
    print("number of repetitions is detected is: ", repetitions)
    print("number of sensors detected is: ", sensors)

    # gesture_data = []


    # all gestures types
    for i in range(gestures):

        repetition_data = []

        # all repetitions of a specific type
        for j in range(repetitions): # shape (28,)

            feature_matrix = np.zeros((18, sensors))

            # all channels 
            for k in range(sensors): #llop through the 24 channels

                # feature Extraction
                emg_features, features_names = features_estimation(set_of_all_gesture_by_subject[j][i][:,k],  
                                                                   "Gesture " + str(i+1) + " repetition " + str(j+1) + 
                                                                   " channel " + str(k+1)  , sampling_rate, window_size, 
                                                                   step_size)
                
                print(type(emg_features))

                
                
                convert to numpy and flatten
                emg_features = emg_features.to_numpy().flatten()

                # we store in the feature matrix
                feature_matrix[:, k] = emg_features

            # store in feature in repetition list
            repetition_data.append(feature_matrix) 

        # srtore in gesture list
        gesture_data.append(repetition_data)




    # save
    output_dir = "E:\Chris\putEMG Project\Data\3_extracted_features"
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

    output_path = os.path.join(output_dir, 'extracted_features_subject'+ str(sub_num)+ '.mat')
    # output_data = {'features': features_for_gesture_type}
    # scipy.io.savemat(output_path, output_data)

    scipy.io.savemat(output_path, {'gestures': gesture_data})




In [ ]:
num_of_subject = 54

# for subject in range(1,num_of_subject):
#     if subject not in [0,1,2,21,28,32,37,40,41,44,52]:
#         file_path = "E:\Chris\putEMG Project\Data\\2_uniformized_data\processed_gestures_subject"+ str(subject)+ ".mat"
#         print(file_path)
#         extract_features(file_path, subject, 5120, 15375, 15376)


file_path = "E:\Chris\putEMG Project\Data\\2_uniformized_data\processed_gestures_subject"+ str(num_of_subject-1)+ ".mat"
file_path1 = "E:\Chris\putEMG Project\Code\Matlab\AllFeaturesTableMaker.m"
print(file_path)
extract_features(file_path, num_of_subject, 5120, 15375, 15376)

E:\Chris\putEMG Project\Data\2_uniformized_data\processed_gestures_subject53.mat
in the function
number of gestures detected is:  7
number of repetitions is detected is:  28
number of sensors detected is:  24
EMG features were from channel Gesture 1 repetition 1 channel 1 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 1 channel 2 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 1 channel 3 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 1 channel 4 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 1 channel 5 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 1 channel 6 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 

e:\Chris\putEMG Project\Code\python\feature_extraction.py:73: RuntimeWarning: divide by zero encountered in log10
  log_detector.append(np.exp(np.sum(np.log10(np.absolute(x))) / frame))


EMG features were from channel Gesture 1 repetition 2 channel 13 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 2 channel 14 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 2 channel 15 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 2 channel 16 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 2 channel 17 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 2 channel 18 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 2 channel 19 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 2 channel 20 extracted successfully
<class 'pandas.core.frame.Data

e:\Chris\putEMG Project\Code\python\feature_extraction.py:188: RuntimeWarning: overflow encountered in scalar multiply
  can = X[cont] * X[cont + 1]


EMG features were from channel Gesture 1 repetition 8 channel 14 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 8 channel 15 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 8 channel 16 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 8 channel 17 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 8 channel 18 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 8 channel 19 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 8 channel 20 extracted successfully
<class 'pandas.core.frame.DataFrame'>
EMG features were from channel Gesture 1 repetition 8 channel 21 extracted successfully
<class 'pandas.core.frame.Data

KeyboardInterrupt: 